<a href="https://colab.research.google.com/github/clementeaf/codingDojo-DS/blob/main/machinelearning/week3/Machine_Learning_Exam_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

### 1.- Cargar los datos

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/drug200.csv')


### 2.- Revisar datos faltantes por columna

In [ ]:
# Obtener los nombres de las columnas
column_names = df.columns

# Obtener la cantidad de datos faltantes en cada columna
missing_values = df.isna().sum()

# Crear un DataFrame con los resultados
results = pd.DataFrame({"Missing values": missing_values})

# Imprimir el DataFrame
print(results)

### 3.- Verificar si los datos existentes corresponden al tipo de datos por columna que indica el diccionario facilitado en el enunciado

In [ ]:
# Diccionario:
data_types = {
    'Drug': object,
    'Age': int,
    'Sex': object,
    'BP': object,
    'Cholesterol': object,
    'Na_to_K': float
}

# Lista para almacenar las columnas con datos incorrectos
incorrect_data_columns = []

# Verificar los tipos de dato en cada columna
for column, dtype in data_types.items():
    if df[column].dtype != dtype:
        incorrect_data_columns.append(column)

# Mostrar las columnas con datos incorrectos (si las hay)
if len(incorrect_data_columns) > 0:
    print("Las siguientes columnas tienen datos incorrectos:")
    for column in incorrect_data_columns:
        print(f"- {column}")
else:
    print("Todos los datos cumplen con los tipos de dato correspondientes.")

### 4.- Verificación de varlores categóricos

In [29]:
# Obtener las columnas con tipos de dato categóricos
categorical_columns = df.select_dtypes(include=['object']).columns

# Mostrar las variables categóricas
print("Variables categóricas en el conjunto de datos:")
for column in categorical_columns:
    print(f"- {column}")

Variables categóricas en el conjunto de datos:
- Sex
- BP
- Cholesterol
- Drug


### 5.- Identificar la cantidad de categórias únicas en cada columna categórica:

In [27]:
for column in categorical_columns:
    unique_categories = df[column].nunique()
    print(f"{column}: {unique_categories} categorías únicas")

Sex: 2 categorías únicas
BP: 3 categorías únicas
Cholesterol: 2 categorías únicas


### 6.- Distribución de categŕias

In [ ]:
# Función para mostrar las etiquetas con cantidades y porcentajes
def show_labels(ax):
    for p in ax.patches:
        ax.annotate(f'{p.get_height()}\n({p.get_height() / len(df) * 100:.1f}%)',
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', fontsize=10, color='black', xytext=(0, 5), textcoords='offset points')

# Visualizar la distribución de categorías en "Sex"
plt.figure(figsize=(8, 6))
ax = sns.countplot(data=df, x='Sex')
plt.title('Distribución de Género')
plt.xlabel('Género')
plt.ylabel('Frecuencia')
show_labels(ax)
plt.show()

# Visualizar la distribución de categorías en "BP"
plt.figure(figsize=(8, 6))
ax = sns.countplot(data=df, x='BP')
plt.title('Distribución de Niveles de Presión Arterial')
plt.xlabel('Niveles de Presión Arterial')
plt.ylabel('Frecuencia')
show_labels(ax)
plt.show()

# Visualizar la distribución de categorías en "Cholesterol"
plt.figure(figsize=(8, 6))
ax = sns.countplot(data=df, x='Cholesterol')
plt.title('Distribución de Niveles de Colesterol')
plt.xlabel('Niveles de Colesterol')
plt.ylabel('Frecuencia')
show_labels(ax)
plt.show()

# Visualizar la distribución de categorías en "Drug"
plt.figure(figsize=(10, 6))
ax = sns.countplot(data=df, x='Drug')
plt.title('Distribución de Tipos de Fármacos (Objetivo)')
plt.xlabel('Tipo de Fármaco')
plt.ylabel('Frecuencia')
show_labels(ax)
plt.xticks(rotation=45)
plt.show()

# Calcular y mostrar la diferencia en las distribuciones
differences = {
    'Sex': df['Sex'].value_counts().diff().dropna().mean(),
    'BP': df['BP'].value_counts().diff().dropna().mean(),
    'Cholesterol': df['Cholesterol'].value_counts().diff().dropna().mean(),
    'Drug': df['Drug'].value_counts().diff().dropna().mean()
}

print("Diferencia promedio en las distribuciones:")
for column, diff in differences.items():
    if column == 'Drug':
        if diff > 45:  # Ajusta este umbral según consideres apropiado
            diff_label = "alta"
        elif diff > 15:
            diff_label = "media"
        else:
            diff_label = "baja"
    else:
        if diff > 5:
            diff_label = "alta"
        elif diff > 2:
            diff_label = "media"
        else:
            diff_label = "baja"
    print(f"- {column}: {diff:.2f} ({diff_label} diferencia)")

### 7.- Relación con la Variable Objetivo

In [ ]:
# Crear gráficos de barras para la relación entre cada variable categórica y la variable objetivo
plt.figure(figsize=(15, 6))

plt.subplot(1, 3, 1)
sns.countplot(data=df, x='Sex', hue='Drug')
plt.title('Relación entre Género y Tipo de Fármaco')
plt.xlabel('Género')
plt.ylabel('Frecuencia')

plt.subplot(1, 3, 2)
sns.countplot(data=df, x='BP', hue='Drug')
plt.title('Relación entre Niveles de Presión Arterial y Tipo de Fármaco')
plt.xlabel('Niveles de Presión Arterial')
plt.ylabel('Frecuencia')

plt.subplot(1, 3, 3)
sns.countplot(data=df, x='Cholesterol', hue='Drug')
plt.title('Relación entre Niveles de Colesterol y Tipo de Fármaco')
plt.xlabel('Niveles de Colesterol')
plt.ylabel('Frecuencia')

plt.tight_layout()
plt.show()


### .8- Correlación
#### Uso de coeficiente de Cramer's V (mide la fuerza y la dirección de la asosiación entre dos variables categóricas)

In [ ]:
!pip install researchpy

In [24]:
import researchpy as rp
from scipy.stats import chi2_contingency

In [25]:
# Calcular la correlación entre variables categóricas y la variable objetivo
correlation_results = []

for column in categorical_columns:
    table, results = rp.crosstab(df[column], df['Drug'], test='chi-square')
    cramers_v = results['results'][2]
    correlation_results.append((column, cramers_v))

# Mostrar los resultados de correlación
for column, correlation in correlation_results:
    print(f"Correlación entre {column} y Drug: {correlation:.4f}")

Correlación entre Sex y Drug: 0.1029
Correlación entre BP y Drug: 0.5984
Correlación entre Cholesterol y Drug: 0.3131


### 9.- Análisis de Frecuencia

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Drug', order=df['Drug'].value_counts().index)
plt.title('Distribución de Tipos de Fármacos')
plt.xlabel('Tipo de Fármaco')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45)
plt.show()

## 10.- Análisis preliminar:

#### Distribución de categorías: las variables categóricas "Sex", "BP", "Cholesterol" y "Drug" tiene un valor razonable de categorías únicas y no parece haber un orden inherente en las categorías

#### Relacion con la Variable Objetivo: Se analizó la relación entre las variables categóricas y la variable objetivo, observando su ditribución en virtud a los diferentes fármacos

#### Correlación y anaálisis de frecuencia: Uso del coeficiente Cramer's V, aun cuando las correlaciones no son extremadamente altas, persiste evidencia de que las variables categóricas están relacionadas con el tipo de fármaco

#### Diferencia en distribuciones: Aún cuando la diferencia en las distribuciones pueden variar, en general, se avista que estas son sustanciales en algunos casos, lo que respalda la idea de que estas variables categóricas podrían ser relevantes para la predicción.

#### Conforme a lo anteior, la docidificaciónm "on-hot" se aprecia como la opción más sólida para capturar las relaciones y difrencias entre las categorías en las variables categóricas.

#### De todos modos, se procederá e evaluar esta premisa conforme a la prueba de modelo para confirmar el tipo de codificación

### 11.- Codificación On-Hot

In [ ]:
# Realizar la codificación one-hot para las variables categóricas
df_encoded = pd.get_dummies(df, columns=['Sex', 'BP', 'Cholesterol', 'Drug'], drop_first=True)

# Mostrar las primeras filas del DataFrame codificado
print(df_encoded.head())

# Modelos de Predicción

### 12.- Random Forest

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Definir las características (X) y la variable objetivo (y)
X = df_encoded.drop(['Drug_drugA', 'Drug_drugB', 'Drug_drugC', 'Drug_drugX'], axis=1)
y = df_encoded[['Drug_drugA', 'Drug_drugB', 'Drug_drugC', 'Drug_drugX']]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear y entrenar un modelo de Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Predecir con el modelo de Random Forest
y_pred_rf = rf_model.predict(X_test_scaled)

# Evaluar el rendimiento del modelo de Random Forest
accuracy_rf = accuracy_score(y_test, y_pred_rf)
classification_report_rf = classification_report(y_test, y_pred_rf, zero_division=1)

print(f"Accuracy del modelo de Random Forest: {accuracy_rf:.4f}")
print("Reporte de clasificación del modelo de Random Forest:")
print(classification_report_rf)

#### 12.1 Random Forest Matriz de confusión

In [ ]:
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test.values.argmax(axis=1), y_pred_rf.argmax(axis=1))

# Etiquetas de las clases
class_labels = ['Drug_drugA', 'Drug_drugB', 'Drug_drugC', 'Drug_drugX']

# Crear una figura y un eje
plt.figure(figsize=(8, 6))
ax = plt.subplot()

# Generar la matriz de confusión con colores
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels, ax=ax)

# Configurar etiquetas de los ejes
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
ax.set_title('Matriz de Confusión - Random Forest')

# Mostrar la leyenda de colores
cbar = ax.collections[0].colorbar
cbar.set_ticks([0, 1, 2, 3])
cbar.set_ticklabels(class_labels)

# Mostrar la figura
plt.show()

### 13.- Regresión Logística Multiclase

In [ ]:
from sklearn.linear_model import LogisticRegression

# Cargar los datos y codificar las variables categóricas
label_encoders = {}
categorical_columns = ['Sex', 'BP', 'Cholesterol', 'Drug']
for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Definir características (X) y variable objetivo (y)
X = df.drop('Drug', axis=1)
y = df['Drug']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear el modelo de Regresión Logística Multiclase
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)

# Entrenar el modelo
logistic_model.fit(X_train_scaled, y_train)

# Predecir con el modelo
y_pred = logistic_model.predict(X_test_scaled)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
classification_report_logistic = classification_report(y_test, y_pred, zero_division=1)

print(f"Accuracy del modelo de Regresión Logística: {accuracy:.4f}")
print("Reporte de clasificación del modelo de Regresión Logística:")
print(classification_report_logistic)

#### 13.1 .- Matriz de confusión para Regresión Logística Multiclase

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Obtener las clases únicas
unique_classes = sorted(y_test.unique())

# Crear una figura y un eje
plt.figure(figsize=(10, 8))
ax = plt.subplot()

# Generar la matriz de confusión con colores
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=unique_classes, yticklabels=unique_classes, ax=ax)

# Configurar etiquetas de los ejes
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
ax.set_title('Matriz de Confusión - Regresión Logística Multiclase')

# Mostrar la figura
plt.show()

### 14.- K-Nearest Neighbors (KNN)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Definir características (X) y variable objetivo (y)
X = df_encoded.drop(['Drug_drugA', 'Drug_drugB', 'Drug_drugC', 'Drug_drugX'], axis=1)
y = df_encoded[['Drug_drugA', 'Drug_drugB', 'Drug_drugC', 'Drug_drugX']]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear y entrenar un modelo de K-Nearest Neighbors
knn_model = KNeighborsClassifier(n_neighbors=10, weights='distance', p=2)
knn_model.fit(X_train_scaled, y_train)

# Predecir con el modelo de K-Nearest Neighbors
y_pred_knn = knn_model.predict(X_test_scaled)

# Evaluar el rendimiento del modelo de K-Nearest Neighbors
accuracy_knn = accuracy_score(y_test, y_pred_knn)
classification_report_knn = classification_report(y_test, y_pred_knn, zero_division=1)

# Análisis del rendimiento del modelo KNN
print(f"Accuracy del modelo de K-Nearest Neighbors: {accuracy_knn:.4f}")
print("Reporte de clasificación del modelo de K-Nearest Neighbors:")
print(classification_report_knn)
print("\n")

# Interpretación de los resultados
print("Interpretación de los resultados:")
print(f"Accuracy: El accuracy del modelo KNN en el conjunto de prueba es del {accuracy_knn:.4f}, lo que significa que aproximadamente el {accuracy_knn * 100:.1f}% de las predicciones del modelo coinciden con las etiquetas reales en el conjunto de prueba. Un accuracy alto generalmente es deseado, pero debes considerar el contexto y el equilibrio de clases.")
print("\n")
print("Precision, Recall y F1-Score: Estas métricas son útiles para evaluar el rendimiento del modelo en cada clase individualmente. En general, el modelo tiene buenas precisiones para la mayoría de las clases. Sin embargo, para la clase '2' (Drug_drugC), la precisión es 1.00 y el recall es 0.80, lo que podría indicar que el modelo puede estar cometiendo algunos falsos negativos en esa clase. La clase '0' (Drug_drugA) también tiene un recall de 0.83, lo que sugiere cierta dificultad en identificar todas las instancias de esa clase. Es importante considerar que estas métricas pueden variar según las preferencias y prioridades del problema.")


#### 14.1.- Matriz de confusión para: K-Nearest Neighbors (KNN)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Obtener la matriz de confusión
conf_matrix = confusion_matrix(y_test.values.argmax(axis=1), y_pred_knn.argmax(axis=1))

# Etiquetas de las clases
class_labels = ['Drug_drugA', 'Drug_drugB', 'Drug_drugC', 'Drug_drugX']

# Crear un mapa de calor para visualizar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Matriz de Confusión')
plt.xlabel('Predicciones')
plt.ylabel('Valores Verdaderos')
plt.show()


### 15.- Validación cruzada: comparar y evaluar el rendimiento de los diferentes modelos en términos de accuracy y otros métricas de evaluación.

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score

# Definir las características (X) y la variable objetivo (y)
X = df_encoded.drop(['Drug_1', 'Drug_2', 'Drug_3', 'Drug_4'], axis=1)
y = df_encoded[['Drug_1', 'Drug_2', 'Drug_3', 'Drug_4']]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelo Random Forest original
rf_model_original = RandomForestClassifier(random_state=42)
rf_model_original.fit(X_train_scaled, y_train)
y_pred_rf_original = rf_model_original.predict(X_test_scaled)

# Evaluar el rendimiento del modelo Random Forest original
accuracy_rf_original = accuracy_score(y_test, y_pred_rf_original)
classification_report_rf_original = classification_report(y_test, y_pred_rf_original, zero_division=1)

print("Resultados Random Forest Original:")
print(f"Accuracy del modelo de Random Forest: {accuracy_rf_original:.4f}")
print("Reporte de clasificación del modelo de Random Forest:")
print(classification_report_rf_original)
print("\n")

# Modelo Random Forest con validación cruzada
rf_model_crossval = RandomForestClassifier(random_state=42)

# Realizar validación cruzada
cv_scores = cross_val_score(rf_model_crossval, X_train_scaled, y_train, cv=5, scoring='accuracy')

print("Resultados Random Forest con Validación Cruzada:")
print("Accuracy en validación cruzada:")
print(cv_scores)
print(f"Accuracy promedio en validación cruzada: {cv_scores.mean():.4f}")
print("\n")

# Explorar otros modelos: Regresión Logística y K-Nearest Neighbors
logistic_model = LogisticRegression(max_iter=1000, random_state=42)

# Entrenar un modelo para cada etiqueta
y_pred_logistic = []

for label in y.columns:
    y_train_label = y_train[label]
    logistic_model.fit(X_train_scaled, y_train_label)
    y_pred_label = logistic_model.predict(X_test_scaled)
    y_pred_logistic.append(y_pred_label)

# Convertir la lista de predicciones a un array
y_pred_logistic = np.array(y_pred_logistic).T

# Calcular el F1-score macro y ponderado para el modelo de Regresión Logística
f1_macro_logistic = f1_score(y_test, y_pred_logistic, average='macro')
f1_weighted_logistic = f1_score(y_test, y_pred_logistic, average='weighted')

# Imprimir resultados por etiqueta
for i, label in enumerate(y.columns):
    precision = precision_score(y_test[label], y_pred_logistic[:, i])
    recall = recall_score(y_test[label], y_pred_logistic[:, i])
    f1 = f1_score(y_test[label], y_pred_logistic[:, i])

    print(f"Etiqueta: {label}")
    print(f"Precisión: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print("\n")

# Explorar el modelo K-Nearest Neighbors
knn_model = KNeighborsClassifier(n_neighbors=10, weights='distance', p=2)
knn_model.fit(X_train_scaled, y_train)
y_pred_knn = knn_model.predict(X_test_scaled)

# Calcular el accuracy para el modelo de K-Nearest Neighbors
accuracy_knn = accuracy_score(y_test, y_pred_knn)
classification_report_knn = classification_report(y_test, y_pred_knn, zero_division=1)

print("Resultados K-Nearest Neighbors:")
print(f"Accuracy del modelo de K-Nearest Neighbors: {accuracy_knn:.4f}")
print("Reporte de clasificación del modelo de K-Nearest Neighbors:")
print(classification_report_knn)


# Informe rendimiento de Modelos de Clasificación

### Resumen:

#### Este informe detalla el proceso de evaluación y comparación de dos modelos de clasificación: Random Forest y K-Nearest Neighbors (KNN). El objetivo es comprender y analizar el rendimiento de estos modelos en la tarea de clasificar medicamentos basados en características médicas.

### Modelo Random Forest Original (Sin ajustes de hiperparámetros)

#### Se entrenó con el conjunto de entrenamiento escalado y se evaluó su rendimiento en el conjunto de prueba. Se calcularon métricas como: Exactitud (accuracy), precisión, recuperación y F1-score, para cada clase y se generó un reporte de clasificación

### Modelo Random Forest con Validación cruzada:

#### Se evaluó mediante validación cruzada para obtener una medida mas robusta del rendimiento. Se calculó el promedio de exactitud en las particiones de validación.

### Modelo de Regresión Logística:

### Se transformaron las etiquetas a formato binario y se ajustó el modelo. Se calcularon las probabilidades de predicción y se estableció un umbral para obtener predicciones binarias. Se calcularon F1-score macro y ponderado para este modelo

### Resultados K-Nearest Neighbors

#### Se entrenó con el conjunto de entrenamiento escalado y se evaluó su rendimiento en el conjunto de prueba. Se calcularon métricas de clasificación como: precisión, recuperación y F1-score para cada clase.

#### EL objetivo de este análisis es evaluar la capacidad de los modelos para predecir correctamente las etiquetas de los medicamentos basados en las características médicas proporcionadas. La precisión, recuperación y F1-score, para medir el rendimiento de los modelos en términos de predicciones correctas y el equilibrio entre precisión y recuperación.

### Conclusiones y Observaciones:

#### 1.- El modelo Random Forest Original muestra un rendimiento excelente con una exactitud del 100%, sin embargo, se necesita precaución ya que el modelo puede estar sobreajustando los datos.

#### 2.- El modelo random Forest con Validación cruzada, proporciona una medida mas realista del rendimiento y muestra una exactitud primedio del 94,37%, lo que sugiere un buen desempeño generalizado.

#### 3.- La evaluación por etiqueta en ambos modelos de Random Forest, revela que el modelo es más fuerte en la predicción de "Drug_3", con una precisión, recuperación y F1-score del 100%.

#### 4.- El modelo de Regresión Logística muestra buenos resultados, con F1-score macro y ponderado en línea con el modelo Random Forest.

#### 5.- El modelo K-nearest neighbors alcanza una exactitud del 92.5%, con un rendimiento sólido en la mayoría de las métricas de clasificación.

### Posibilidades de Iteración y Rectificación

#### 1.- Considerar ajustar hiperparámetros en el modelo Random Forest para evitar el sobreajuste.

#### 2.- Explorar otros algoritmos de clasificación tales como: Support Vector Machines o Grgadient boosting.

#### 3.- Realizar una búsqueda más exhaustiva de hiperparámetros para cada modelo

#### 4.- Relizar un análisis más profundo de las características para entender su importancia en la clasificación.

### Elección del modelo

### Dependerá de los objetivos y requisitos específicos, en caso de que la prioridad sea la precisión, el modelo Random Forest con Validación Cruzada es una opción sólida. Si se necesita un modelo interpretable, la Regresión Logística podría ser preferible. En general Random Forest y KNN, ofrecen rendimiento respetable y podrían ser considerados en diferentes contextos.
